$$\mu_0 = 4\pi\times10^{-7}$$
$$\varepsilon_0=(4\pi)^{-1}c^{-2}\times10^7$$

In [ ]:
%%writefile pyfdtd3d/constants.py
import numpy as np
c = 299792458 # [m/s]
MU0 = 4 * np.pi * 1e-7
EP0 = 1 / MU0 / c / c

In [ ]:
%%writefile pyfdtd3d/base.py
import numpy as np

class Grid(object):
    pass

class Calculator(object):
    def calculate(self):
        pass

class Probe(object):
    def get(self):
        pass
    
class TimeSeries(object):
    dt = property(lambda self: self._dt)
    ratio = property(lambda self: self._ratio)
    current_time = property(lambda self: self._current_time)
    def __init__(self, dt, ratio=.5):
        self._current_time = 0.
        self._dt = dt # [ns]
        self._ratio = ratio
    def progress(self):
        self._current_time += self.ratio * self.dt

def sin3(current_time,freq): # time: float, freq: float
    omg0 = 2 * np.pi * freq # 正弦波源の各周波数
    tau0 = .5 / freq        # パルス持続半周期
    if current_time > 2. * tau0:
        v = 0.
    else:
        v = np.sin(omg0 * current_time) ** 3
    return v

def vsrc(voltage, current, delta, resister=50):
    return - (voltage - resister * current) / delta

In [ ]:
%%writefile pyfdtd3d/structuredpoints.py
from pyfdtd3d.constants import EP0, MU0
from pyfdtd3d.base import TimeSeries
import pyfdtd3d.base
import numpy as np
import gc

class Grid(pyfdtd3d.base.Grid):
    dimensions = property(lambda self: self._dimensions)
    spacing = property(lambda self: self._spacing)
    origin = property(lambda self: self._origin)
    def __init__(self, dimensions, spacing, origin=(0, 0, 0)):
        pyfdtd3d.base.Grid.__init__(self)
        self._dimensions = dimensions # (nx, ny, nz)
        self._spacing = spacing # (dx, dy, dz)
        self._origin = origin

class Yee(object):
    ex = property(lambda self: self._ex)
    ey = property(lambda self: self._ey)
    ez = property(lambda self: self._ez)
    hx = property(lambda self: self._hx)
    hy = property(lambda self: self._hy)
    hz = property(lambda self: self._hz)
    e = property(lambda self: (self.ex, self.ey, self.ez))
    h = property(lambda self: (self.hx, self.hy, self.hz))
    grid = property(lambda self: self._grid)
    def __init__(self, grid):
        self._grid = g = grid
        nx, ny, nz = g.dimensions
        self._ex = np.zeros((nx, ny+1, nz+1))
        self._ey = np.zeros((nx+1, ny, nz+1))
        self._ez = np.zeros((nx+1, ny+1, nz))
        self._hx = np.zeros((nx+1, ny, nz))
        self._hy = np.zeros((nx, ny+1, nz))
        self._hz = np.zeros((nx, ny, nz+1))

class Material(object):
    er = property(lambda self: self._er)
    def __init__(self, nx, ny, nz):
        self._er = np.ones((nx, ny, nz))

class HCalc(pyfdtd3d.base.Calculator):
    def __init__(self, yee, time):
        pyfdtd3d.base.Calculator.__init__(self)
        self._yee = y = yee
        dx, dy, dz = y.grid.spacing
        self._const = time.dt / MU0 / np.array((dx, dy, dz))
        self.time = time
    def calculate_(self):
        ex, ey, ez = self._yee.e
        hx, hy, hz = self._yee.h
        cx, cy, cz = self._const
        hx[ :  , :  , :  ] -= cy * (ez[ :  ,1:  , :  ] - ez[ :  , :-1, :  ])\
                            - cz * (ey[ :  , :  ,1:  ] - ey[ :  , :  , :-1])
        hy[ :  , :  , :  ] -= cz * (ex[ :  , :  ,1:  ] - ex[ :  , :  , :-1])\
                            - cx * (ez[1:  , :  , :  ] - ez[ :-1, :  , :  ])
        hz[ :  , :  , :  ] -= cx * (ey[1:  , :  , :  ] - ey[ :-1, :  , :  ])\
                            - cy * (ex[ :  ,1:  , :  ] - ex[ :  , :-1, :  ])       
    def calculate(self):
        ex, ey, ez = self._yee.e
        hx, hy, hz = self._yee.h
        dx, dy, dz = self._yee.grid.spacing
        dt = self.time.dt
        hx[ :  , :  , :  ] -= dt / MU0 / dy * (ez[ :  ,1:  , :  ] - ez[ :  , :-1, :  ])\
                            - dt / MU0 / dz * (ey[ :  , :  ,1:  ] - ey[ :  , :  , :-1])
        hy[ :  , :  , :  ] -= dt / MU0 / dz * (ex[ :  , :  ,1:  ] - ex[ :  , :  , :-1])\
                            - dt / MU0 / dx * (ez[1:  , :  , :  ] - ez[ :-1, :  , :  ])
        hz[ :  , :  , :  ] -= dt / MU0 / dx * (ey[1:  , :  , :  ] - ey[ :-1, :  , :  ])\
                            - dt / MU0 / dy * (ex[ :  ,1:  , :  ] - ex[ :  , :-1, :  ])       

class ECalc(pyfdtd3d.base.Calculator):
    def __init__(self, yee, time, material):
        pyfdtd3d.base.Calculator.__init__(self)
        self._yee = y = yee
        dx, dy, dz = y.grid.spacing
        er = material.er
        erx = (er[ :  , :-1, :-1] +
               er[ :  , :-1,1:  ] +
               er[ :  ,1:  , :-1] +
               er[ :  ,1:  ,1:  ]) / 4.
        ery = (er[ :-1, :  , :-1] +
               er[1:  , :  , :-1] +
               er[ :-1, :  ,1:  ] +
               er[1:  , :  ,1:  ]) / 4.
        erz = (er[ :-1, :-1, :  ] +
               er[ :-1,1:  , :  ] +
               er[1:  , :-1, :  ] +
               er[1:  ,1:  , :  ]) / 4.
        e = np.array((erx, erx, ery, ery, erz, erz))
        d = np.array((dy, dz, dz, dx, dx, dy))
        self._const = time.dt / EP0 / e / d # cxy, cxz, cyz, cyx, czx, czy
        self.time = time
        self._er = erx, ery, erz
    def calculate_(self):
        ex, ey, ez = self._yee.e
        hx, hy, hz = self._yee.h
        cxy, cxz, cyz, cyx, czx, czy = self._const
        ex[ :  ,1:-1,1:-1] += cxy * (hz[ :  ,1:  ,1:-1] - hz[ :  , :-1,1:-1])\
                            - cxz * (hy[ :  ,1:-1,1:  ] - hy[ :  ,1:-1, :-1])
        ey[1:-1, :  ,1:-1] += cyz * (hx[1:-1, :  ,1:  ] - hx[1:-1, :  , :-1])\
                            - cyx * (hz[1:  , :  ,1:-1] - hz[ :-1, :  ,1:-1])
        ez[1:-1,1:-1, :  ] += czx * (hy[1:  ,1:-1, :  ] - hy[ :-1,1:-1, :  ])\
                            - czy * (hx[1:-1,1:  , :  ] - hx[1:-1, :-1, :  ])
    def calculate(self):
        ex, ey, ez = self._yee.e
        hx, hy, hz = self._yee.h
        dx, dy, dz = self._yee.grid.spacing
        dt = self.time.dt
        erx, ery, erz = self._er
        ex[ :  ,1:-1,1:-1] += dt / EP0 / erx / dy * (hz[ :  ,1:  ,1:-1] - hz[ :  , :-1,1:-1])\
                            - dt / EP0 / erx / dz * (hy[ :  ,1:-1,1:  ] - hy[ :  ,1:-1, :-1])
        ey[1:-1, :  ,1:-1] += dt / EP0 / ery / dz * (hx[1:-1, :  ,1:  ] - hx[1:-1, :  , :-1])\
                            - dt / EP0 / ery / dx * (hz[1:  , :  ,1:-1] - hz[ :-1, :  ,1:-1])
        ez[1:-1,1:-1, :  ] += dt / EP0 / erz / dx * (hy[1:  ,1:-1, :  ] - hy[ :-1,1:-1, :  ])\
                            - dt / EP0 / erz / dy * (hx[1:-1,1:  , :  ] - hx[1:-1, :-1, :  ])


class CurrentX(pyfdtd3d.base.Probe):
    def __init__(self, yee):
        pyfdtd3d.base.Probe.__init__(self)
        self._yee = yee
    def get(self, pos):
        hx, hy, hz = self._yee.h
        dx, dy, dz = self._yee.grid.spacing
        i, j, k = pos
        return (hz[i,j,k] - hz[i  ,j-1,k  ]) * dz + (hy[i  ,j  ,k-1] - hy[i,j,k]) * dy

class CurrentY(pyfdtd3d.base.Probe):
    def __init__(self, yee):
        pyfdtd3d.base.Probe.__init__(self)
        self._yee = yee
    def __getitem__(self, pos):
        hx, hy, hz = self._yee.h
        dx, dy, dz = self._yee.grid.spacing
        i, j, k = pos
        return (hx[i,j,k] - hx[i  ,j  ,k-1]) * dx + (hz[i-1,j  ,k  ] - hz[i,j,k]) * dz

class CurrentZ(pyfdtd3d.base.Probe):
    def __init__(self, yee):
        pyfdtd3d.base.Probe.__init__(self)
        self._yee = yee
    def __getitem__(self, pos):
        hx, hy, hz = self._yee.h
        dx, dy, dz = self._yee.grid.spacing
        i, j, k = pos
        return (hy[i,j,k] - hy[i-1,j  ,k  ]) * dy + (hx[i  ,j-1,k  ] - hx[i,j,k]) * dx
        

In [ ]:
%%writefile pyfdtd3d/structuredpoints_cupy.py
from pyfdtd3d.structuredpoints import *
import cupy as cp

#class FieldCP(Field):
#    def alloc(self, nx, ny, nz):
#        self.value = cp.zeros((nx, ny, nz))

#class FieldsCP(Fields):
#    def __init__(self):
#        Fields.__init__(self)
#        self.values = dict([(k, FieldCP()) for k in ['ex', 'ey', 'ez', 'hx', 'hy', 'hz']])


In [ ]:
%%writefile pyfdtd3d/structuredpoints_ui.py
from IPython.display import display
import ipywidgets as widgets
from bokeh.io import output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.models import LinearColorMapper
import threading
import abc
import numpy as np

class Solver(object):
    def __init__(self, **kw):
        self.parameters = {}
        self.config(**kw)
        self.yee = None
        self.time = None
    def config(self, **kw):
        for k in kw:
            self.parameters[k] = kw[k]
    def reset(self):
        self.prepare()
    def calculate(self):
        pass
    def prepare(self):
        pass

class ControlPanel(object):
    def __init__(self, plotter, solver):
        self._plotter = plotter
        self._solver = solver
        #
        self.counter_int = widgets.IntText(value=0, description='Current Count:', disabled=True)
        self.counter_limit_int = widgets.IntText(value=1000, description='Count Limit:', disabled=False)
        display(widgets.HBox([self.counter_int, self.counter_limit_int]))
        #
        self.current_time = widgets.FloatText(value=0, description='Current Time:', disabled=True)
        self.plot_every_int = widgets.IntText(value=10, description='Plot Every:', disabled=False)
        display(widgets.HBox([self.current_time, self.plot_every_int]))
        #
        self.calc_button = widgets.Button(description='Calculate')
        self.pause_button = widgets.Button(description='Pause')
        self.reset_button = widgets.Button(description='Reset')
        display(widgets.HBox([self.calc_button, self.pause_button, self.reset_button]))
        #
        self.calc_button.on_click(self.calculate)
        self.pause_button.on_click(self.pause)
        self.reset_button.on_click(self.reset)
        #
        self._evt_run = threading.Event()
        self._evt_run.clear()
        self._evt_break = threading.Event()
        self._evt_break.clear()
        self._thread = threading.Thread(target=self.calc_thread)
        self._thread.start()
    def calc_thread(self):
        while True:
            if not (self.counter_int.value < self.counter_limit_int.value):
                self._evt_run.clear()
            self._evt_run.wait()
            self.counter_int.value += 1
            self.current_time.value = self._solver.time.current_time
            self._solver.calculate()
            if not (self.counter_int.value % self.plot_every_int.value):
                self._plotter.push()
    def calculate(self, e):
        self._evt_run.set()
    def pause(self, e):
        self._evt_run.clear()
    def reset(self, e):
        self.pause(e)
        self.counter_int.value = 0
        self._solver.reset()
        self.current_time.value = self._solver.time.current_time
        self._plotter.yee = self._solver.yee
        self._plotter.push()
        #if self._thread.is_alive():
        #    self._evt_break.set()
        #    self.calculate(e)
        #    self._thread.join()
        #    self.pause(e)
        #    self._evt_break.clear()
        #self._thread = threading.Thread(target=self.calc_thread)
        #self._thread.start()
        
class Plotter(object):
    def __init__(self, yee, slices, mapper):
        self.yee = y = yee
        self._slices = s = slices
        ex, ey, ez = y.e
        im = ez.__getitem__(s)
        output_notebook()
        f = figure()
        self._render = f.image(image=[im], x=0, y=0, dw=10, dh=10, color_mapper=mapper)
        self._notebook_handle = show(f, notebook_handle=True)
    def push(self):
        ex, ey, ez = self.yee.e
        self._render.data_source.data['image'] = [ez.__getitem__(self._slices)]
        push_notebook(handle=self._notebook_handle)



# Grid Data

## StructuredPoints
- DIMENSIONS
- ORIGIN
- SPACING

## StructuredGrid

## RectilinearGrid

## Polygonal Grid

## UnstructuredGrid

# Field Data

# References

- http://www.paraview-expert.com/vtk-structured-points/
- https://qiita.com/shohirose/items/dd2bdc4201644455e3a8
